<a href="https://colab.research.google.com/github/Kussil/CVX_Rice_project/blob/main/Text_Data/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
from google.colab import userdata
import pandas as pd

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 127 (delta 31), reused 0 (delta 0), pack-reused 62
Receiving objects: 100% (127/127), 54.32 MiB | 9.18 MiB/s, done.
Resolving deltas: 100% (54/54), done.


## Upload Investment Research Articles into DF

In [3]:
# Import Chevron and Other Majors
invest_df_cvx = pd.read_csv('CVX_Rice_project/Text_Data/Investment Research-CVX.csv')
invest_df_majors = pd.read_csv('CVX_Rice_project/Text_Data/Investment Research-Majors.csv')

# Append dfs
invest_df = pd.concat([invest_df_cvx, invest_df_majors], ignore_index=True)
display(invest_df_cvx.shape)
display(invest_df_majors.shape)
display(invest_df.shape)

# Rename drop, and reorder columns
invest_df = invest_df.rename(columns={'Date/Time': 'Date', 'Company': 'Ticker', 'Headline': 'Article Headline', 'Text': 'Article Text'})
invest_df = invest_df.drop(['Contributor', 'Analyst', 'Pages'], axis=1)
new_order = ['Ticker', 'Date', 'Article Headline', 'Article Text']
invest_df = invest_df.reindex(columns=new_order)
display(invest_df.head())

# Note, still missing ~400 from majors and non majors

(362, 7)

(2200, 7)

(2562, 7)

,Ticker,Date,Article Headline,Article Text
0,CVX,"May 13, 2024 10:05 PM",Chevron Corporation,"Stock Report | March 12, 2022 | NYSE Symbol: C..."
1,CVX,"May 13, 2024 03:29 PM",CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,"Stock Report | August 05, 2023 | NYSE Symbol: ..."
2,CVX,"May 11, 2024 06:00 PM",Chevron Corporation,"Stock Report | March 04, 2023 | NYSE Symbol: C..."
3,CVX,"May 04, 2024 05:49 PM",Chevron Corporation,"Stock Report | October 17, 2020 | NYSE Symbol:..."
4,CVX,"May 01, 2024 10:03 PM",Chevron Corporation,"Stock Report | December 25, 2021 | NYSE Symbol..."


## Upload Proquest News Articles into DF and Clean Data

In [4]:
# Import Chevron and Others
proquest_df_cvx = pd.read_csv('CVX_Rice_project/Text_Data/proquest_newsarticles_CVX.csv')
# proquest_df_others = pd.read_csv('CVX_Rice_project/Text_Data/proquest_newsarticles_all_v2.csv')  # Currently empty for some reason, fix later

# Append dfs
# proquest_df = pd.concat([proquest_df_cvx, proquest_df_others], ignore_index=True)
display(proquest_df_cvx.shape)
# display(proques_df_majors.shape)
# display(proques_df.shape)

# Get rid of this and uncomment above lines once csv is reloaded
proquest_df = proquest_df_cvx

# Rename drop, and reorder columns
proquest_df = proquest_df.rename(columns={'Title': 'Article Headline', 'Full Article Text': 'Article Text'})
proquest_df = proquest_df.drop(['URL'], axis=1)
proquest_df['Ticker'] = "TBD" # Temp column until ticker added
proquest_df = proquest_df.reindex(columns=new_order)
display(proquest_df.head())

(261, 4)

,Ticker,Date,Article Headline,Article Text
0,TBD,"Dec 4, 2020",Chevron General Motors Zoom At Amp T Stocks That,Turn on search term navigationTurn on search t...
1,TBD,"Dec 4, 2020",Chevron Slash Capital Spending,Turn on search term navigationTurn on search t...
2,TBD,"Dec 3, 2020",Chevron Slashes Spending Plans As Coronavirus,Turn on search term navigationTurn on search t...
3,TBD,"Nov 11, 2020",Senators Air Misgivings Over Malampaya Chevron,Relevant content not found within the specifie...
4,TBD,"Oct 29, 2020",Shell Tries Woo Investors With Dividend Raise,Turn on search term navigationTurn on search t...


In [5]:
# Delete rows with missing article text
search_text_1 = 'Failed to load content: Message:'
search_text_2 = 'Relevant content not found within the specified range.'

# Count the number of rows containing either search text
count_rows = proquest_df[proquest_df['Article Text'].str.contains(search_text_1, na=False) |
                         proquest_df['Article Text'].str.contains(search_text_2, na=False)].shape[0]

# Delete the rows containing either search text
proquest_df = proquest_df[~proquest_df['Article Text'].str.contains(search_text_1, na=False) &
                          ~proquest_df['Article Text'].str.contains(search_text_2, na=False)]

# Print the number of rows with missing article text and the new shape of the DataFrame
print(f"Number of rows with missing article text: {count_rows}")
print(f"New DataFrame shape: {proquest_df.shape}")
print()

# Confirm data is good by looking for short article headlines
shortest_headline = proquest_df.loc[proquest_df['Article Headline'].str.len().idxmin(), 'Article Headline']
print(f"The shortest article headline is: '{shortest_headline}'")
print()

# Confirm data is good by looking for short article text
shortest_text = proquest_df.loc[proquest_df['Article Text'].str.len().idxmin(), 'Article Text']
print(f"The shortest article text is: '{shortest_text}'")

Number of rows with missing article text: 98
New DataFrame shape: (163, 4)

The shortest article headline is: 'Chevron Boosts Green Spending'

The shortest article text is: 'Turn on search term navigationTurn on search term navigation
| Jump to first hit
Chevron said it will book $3.5 billion to $4 billion in charges for the fourth quarter, citing challenges tied to regulations in California and previously sold oil-and-gas production assets in the Gulf of Mexico.
Chevron said it will impair a portion of its U.S. upstream assets, mostly in California, because of regulatory challenges in the state that have led to "lower anticipated future investment levels in its business plans." Chevron said it will continue to operate the affected assets for many years.
The company also said companies that agreed to buy Chevron's oil-and-gas production assets in the U.S. Gulf of Mexico have since filed for Chapter 11 bankruptcy protections. Chevron said the obligations for the assets could revert back

In [6]:
# Look for duplicate rows
duplicates = proquest_df[proquest_df.duplicated(subset=['Date', 'Article Headline'], keep=False)]
duplicate_count = duplicates.shape[0]
print(f"Number of duplicate rows: {duplicate_count}")
print(duplicates)
print()

display(proquest_df.head(20))
display(proquest_df.tail(20))

Number of duplicate rows: 0
Empty DataFrame
Columns: [Ticker, Date, Article Headline, Article Text]
Index: []



,Ticker,Date,Article Headline,Article Text
0,TBD,"Dec 4, 2020",Chevron General Motors Zoom At Amp T Stocks That,Turn on search term navigationTurn on search t...
1,TBD,"Dec 4, 2020",Chevron Slash Capital Spending,Turn on search term navigationTurn on search t...
2,TBD,"Dec 3, 2020",Chevron Slashes Spending Plans As Coronavirus,Turn on search term navigationTurn on search t...
4,TBD,"Oct 29, 2020",Shell Tries Woo Investors With Dividend Raise,Turn on search term navigationTurn on search t...
14,TBD,"Oct 2, 2020",Virus Pain Persists Oil Companies,Turn on search term navigationTurn on search t...
15,TBD,"Oct 1, 2020",Pandemic Pain Persists Big Oil Companies Tepid,Turn on search term navigationTurn on search t...
17,TBD,"Aug 24, 2020",Norways Biggest Private Money Manager Exits Exxon,Turn on search term navigationTurn on search t...
18,TBD,"Aug 18, 2020",Chevron Pursues Iraq Oil Project Deals With,Turn on search term navigationTurn on search t...
19,TBD,"Aug 18, 2020",Chevron Pursues Exploration Deal Iraq Ge,Turn on search term navigationTurn on search t...
20,TBD,"Aug 17, 2020",Chevron Pursues Exploration Deal Iraq Ge,Turn on search term navigationTurn on search t...


,Ticker,Date,Article Headline,Article Text
224,TBD,"Feb 3, 2023",Shell Is Very Generous With Buybacks European,Turn on search term navigationTurn on search t...
227,TBD,"Jan 31, 2023",Joe Bidens Big Oil Profits Exxons Shareholders,Turn on search term navigationTurn on search t...
229,TBD,"Jan 27, 2023",Chevron Rides High Oil Prices Record 35 5 Billion,Turn on search term navigationTurn on search t...
230,TBD,"Jan 4, 2023",Exxon Chevron Focus On Oil Projects Americas Two,Turn on search term navigationTurn on search t...
232,TBD,"Jan 3, 2023",Exxon Chevron Focus On Oil Projects Americas Two,Turn on search term navigationTurn on search t...
238,TBD,"Nov 28, 2022",Bidens Man Venezuela U S Eases Sanctions On,Turn on search term navigationTurn on search t...
240,TBD,"Nov 27, 2022",More Venezuelan Oil Unlikely Reach Markets Soon,Turn on search term navigationTurn on search t...
241,TBD,"Nov 26, 2022",Chevron Gets U S License Pump Oil Venezuela Again,Turn on search term navigationTurn on search t...
242,TBD,"Nov 25, 2022",Banking Amp Finance U S Poised Grant Chevron,Turn on search term navigationTurn on search t...
243,TBD,"Nov 24, 2022",U S Poised Grant Chevron License Pump Oil,Turn on search term navigationTurn on search t...


## Concatenate both DF Sources and Export to CSV

In [7]:
# Concatenate
text_df = pd.concat([invest_df, proquest_df], ignore_index=True)
display(text_df.shape)
display(text_df.head())

(2725, 4)

,Ticker,Date,Article Headline,Article Text
0,CVX,"May 13, 2024 10:05 PM",Chevron Corporation,"Stock Report | March 12, 2022 | NYSE Symbol: C..."
1,CVX,"May 13, 2024 03:29 PM",CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,"Stock Report | August 05, 2023 | NYSE Symbol: ..."
2,CVX,"May 11, 2024 06:00 PM",Chevron Corporation,"Stock Report | March 04, 2023 | NYSE Symbol: C..."
3,CVX,"May 04, 2024 05:49 PM",Chevron Corporation,"Stock Report | October 17, 2020 | NYSE Symbol:..."
4,CVX,"May 01, 2024 10:03 PM",Chevron Corporation,"Stock Report | December 25, 2021 | NYSE Symbol..."


In [8]:
# Export as CSV
text_df.to_csv('/content/Consolidated_Text_Data.csv', index=False)

# Note: This export needs to be manually uploaded to Github.  Hopefully will figure out a way to automate this later